# Correct grammar of `questions`

Using GPT-4.

In [35]:
import openai
import json
import pandas as pd
from tqdm.auto import tqdm

In [37]:
prompt_template = '''Below is a list of {}-language questions for the '{}' task, a measure of creativity where {}.

YOUR TASK: CORRECT THE GRAMMAR AS NECESSARY.

The input is as a JSON array of objects with prompt/question keys. The `prompt` is the master reference, the `question` is the potentially erroneous text that may need correction or editing. Don't change the capitalized portion of a question or anything quoted.

Respond in a JSON list of objects, where each object has `prompt`, `oldquestion`, and `fixedquestion` keys. Include all responses, even if they didn't need corrections. Surround the response in triple-backticks.

## Questions to grammar-check

```
{}
```
'''

# 'ara', 'chi', 'dut', 'eng', 'fre', 'ger', 'heb', 'ita', 'pol', 'rus', 'spa'
lang_ref = {
    "ara": "Arabic",
    "chi": "Chinese",
    "dut": "Dutch",
    "eng": "English",
    "fre": "French",
    "ger": "German",
    "heb": "Hebrew",
    "ita": "Italian",
    "pol": "Polish",
    "rus": "Russian",
    "spa": "Spanish"
}

# 'uses', 'completion', 'consequences', 'instances', 'metaphors'
type_ref = {
    "uses": {"name": "Alternate Uses", "desc": "you write down uses for a common object"},
    "completion": {"name": "Complete the Sentence", "desc": "you complete a sentence with an unusual ending"},
    "consequences": {"name": "Consequences", "desc": "you write down consequences of an absurd event"},
    "instances": {"name": "Instances", "desc": "you write down instances of a category"},
    "metaphors": {"name": "Metaphors", "desc": "you write down metaphors for an abstract concept"}
}

,language,type,prompt,question
913,chi,uses,光盘,什么是光盘的一个令人惊讶的用途？
988,chi,uses,冰块,什么是冰块的一个令人惊讶的用途？
1081,chi,uses,勺子,什么是勺子的一个令人惊讶的用途？
1205,chi,uses,南瓜,什么是南瓜的一个令人惊讶的用途？
1286,chi,uses,卫生纸,什么是卫生纸的一个令人惊讶的用途？
...,...,...,...,...
17048,chi,uses,风车,什么是风车的一个令人惊讶的用途？
17151,chi,uses,香蕉,什么是香蕉的一个令人惊讶的用途？
17240,chi,uses,马来貘,什么是马来貘的一个令人惊讶的用途？
17339,chi,uses,鹅卵石,什么是鹅卵石的一个令人惊讶的用途？


In [41]:
data = pd.read_csv('../data/ocsai-all.csv')
unique = data[['language', 'type', 'prompt', 'question']].drop_duplicates()
type_groups = unique.groupby(['type', 'language'])
all_responses = []

for (test_type, language), group in tqdm(type_groups):
    # batch `group` dataframe into subdataframes with 20 rows at a time
    for batch_n in tqdm(range(0, len(group), 20)):    
        batch = group.iloc[batch_n:batch_n+20]
        question_in = batch[['prompt', 'question']].to_json(orient='records', force_ascii=False)
        #print(question_in)
        prompt = prompt_template.format(lang_ref[language], type_ref[test_type]['name'], type_ref[test_type]['desc'], question_in)
        print(prompt)

        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages = [
                {"role": "system", "content": "You are a helpful, multi-lingual editor."},
                {"role": "user", "content": prompt},
            ],
            temperature=0
            )
        content = response['choices'][0]['message']['content']
        content = content.replace('```json', '```').split('```')[1]
        content_parsed = json.loads(content)

        print(content_parsed)
        for obj in content_parsed:
            obj['type'] = test_type
            obj['lang'] = language
            all_responses.append(obj)

/var/folders/k2/vg86l2h54czgjfzlmd6fkz908bdx0t/T/ipykernel_74047/1148676225.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/ocsai-all.csv')


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Below is a list of English-language questions for the 'Complete the Sentence' task, a measure of creativity where you complete a sentence with an unusual ending.

YOUR TASK: CORRECT THE GRAMMAR AS NECESSARY.

The input is as a JSON array of objects with prompt/question keys. The `prompt` is the master reference, the `question` is the potentially erroneous text that may need correction or editing. Don't change the capitalized portion of a question or anything quoted.

Respond in a JSON list of objects, where each object has `prompt`, `oldquestion`, and `fixedquestion` keys. Include all responses, even if they didn't need corrections. Surround the response in triple-backticks.

## Questions to grammar-check

```
[{"prompt":"games","question":"Complete this sentence in a surprising way: \"At a sleepover we...\""},{"prompt":"rain","question":"Complete this sentence in a surprising way: \"It started raining and...\""},{"prompt":"friend phone","question":"Complete this sentence in a surprisi

  0%|          | 0/1 [00:00<?, ?it/s]

...

Below is a list of Polish-language questions for the 'Alternate Uses' task, a measure of creativity where you write down uses for a common object.

YOUR TASK: CORRECT THE GRAMMAR AS NECESSARY.

The input is as a JSON array of objects with prompt/question keys. The `prompt` is the master reference, the `question` is the potentially erroneous text that may need correction or editing. Don't change the capitalized portion of a question or anything quoted.

Respond in a JSON list of objects, where each object has `prompt`, `oldquestion`, and `fixedquestion` keys. Include all responses, even if they didn't need corrections. Surround the response in triple-backticks.

## Questions to grammar-check

```
[{"prompt":"cegła","question":"Jakie jest zaskakujące zastosowanie dla CEGŁA?"},{"prompt":"puszka","question":"Jakie jest zaskakujące zastosowanie dla PUSZKA?"},{"prompt":"sznurek","question":"Jakie jest zaskakujące zastosowanie dla SZNUREK?"}]
```

[{'prompt': 'cegła', 'oldquestion': 'Jakie je

  0%|          | 0/1 [00:00<?, ?it/s]

Below is a list of Russian-language questions for the 'Alternate Uses' task, a measure of creativity where you write down uses for a common object.

YOUR TASK: CORRECT THE GRAMMAR AS NECESSARY.

The input is as a JSON array of objects with prompt/question keys. The `prompt` is the master reference, the `question` is the potentially erroneous text that may need correction or editing. Don't change the capitalized portion of a question or anything quoted.

Respond in a JSON list of objects, where each object has `prompt`, `oldquestion`, and `fixedquestion` keys. Include all responses, even if they didn't need corrections. Surround the response in triple-backticks.

## Questions to grammar-check

```
[{"prompt":"газета","question":"Какое удивительное применение для ГАЗЕТА?"},{"prompt":"деревянная линейка","question":"Какое удивительное применение для ДЕРЕВЯННАЯ ЛИНЕЙКА?"},{"prompt":"картонная коробка","question":"Какое удивительное применение для КАРТОННАЯ КОРОБКА?"}]
```

[{'prompt': 'газ

  0%|          | 0/1 [00:00<?, ?it/s]

Below is a list of Spanish-language questions for the 'Alternate Uses' task, a measure of creativity where you write down uses for a common object.

YOUR TASK: CORRECT THE GRAMMAR AS NECESSARY.

The input is as a JSON array of objects with prompt/question keys. The `prompt` is the master reference, the `question` is the potentially erroneous text that may need correction or editing. Don't change the capitalized portion of a question or anything quoted.

Respond in a JSON list of objects, where each object has `prompt`, `oldquestion`, and `fixedquestion` keys. Include all responses, even if they didn't need corrections. Surround the response in triple-backticks.

## Questions to grammar-check

```
[{"prompt":"ladrillo","question":"¿Cuál es un uso sorprendente para un LADRILLO?"}]
```

[{'prompt': 'ladrillo', 'oldquestion': '¿Cuál es un uso sorprendente para un LADRILLO?', 'fixedquestion': '¿Cuál es un uso sorprendente para un LADRILLO?'}]


In [ ]:
# inspect the fixes and save them
df = pd.DataFrame(all_responses)
for i, row in df[df.oldquestion != df.fixedquestion].iterrows():
    print(row.oldquestion)
    print(row.fixedquestion)
    print()
df.to_csv('../data/question_fixes.csv', index=False)

In [51]:
question_inference_mappings = {
}

for lang, group in df.groupby('lang'):
    question_inference_mappings[lang] = {}
    for test_type, group2 in group.groupby('type'):
        question_inference_mappings[lang][test_type] = {}
        for prompt, group3 in group2.groupby('prompt'):
            question_inference_mappings[lang][test_type][prompt] = group3.iloc[0].fixedquestion

question_inference_mappings

{'ara': {'uses': {'Tin cans': 'ما هو الاستخدام المفاجئ لـ TIN CANS؟'}},
 'chi': {'uses': {'光盘': '光盘有什么令人惊讶的用途？',
   '冰块': '冰块有什么令人惊讶的用途？',
   '勺子': '勺子有什么令人惊讶的用途？',
   '南瓜': '南瓜有什么令人惊讶的用途？',
   '卫生纸': '卫生纸有什么令人惊讶的用途？',
   '发带': '发带有什么令人惊讶的用途？',
   '发簪': '发簪有什么令人惊讶的用途？',
   '台灯': '台灯有什么令人惊讶的用途？',
   '吸管': '吸管有什么令人惊讶的用途？',
   '吹风机': '吹风机有什么令人惊讶的用途？',
   '咖啡': '咖啡有什么令人惊讶的用途？',
   '喇叭': '喇叭有什么令人惊讶的用途？',
   '围巾': '围巾有什么令人惊讶的用途？',
   '图钉': '图钉有什么令人惊讶的用途？',
   '土豆': '土豆有什么令人惊讶的用途？',
   '地图': '地图有什么令人惊讶的用途？',
   '塑料袋': '塑料袋有什么令人惊讶的用途？',
   '墨水': '墨水有什么令人惊讶的用途？',
   '头发': '头发有什么令人惊讶的用途？',
   '夹子': '夹子有什么令人惊讶的用途？',
   '字典': '字典的一个令人惊讶的用途是什么？',
   '小米': '小米的一个令人惊讶的用途是什么？',
   '床单': '床单的一个令人惊讶的用途是什么？',
   '弹弓': '弹弓的一个令人惊讶的用途是什么？',
   '戒指': '戒指的一个令人惊讶的用途是什么？',
   '扇子': '扇子的一个令人惊讶的用途是什么？',
   '手套': '手套的一个令人惊讶的用途是什么？',
   '扑克': '扑克的一个令人惊讶的用途是什么？',
   '报纸': '报纸的一个令人惊讶的用途是什么？',
   '拖鞋': '拖鞋的一个令人惊讶的用途是什么？',
   '擀面杖': '擀面杖的一个令人惊讶的用途是什么？',
   '无花果': '无花果的一个令人惊讶的用途是什么？',
   '易拉罐': '易拉罐的一个令人惊讶的用途是什么？',
   '